# Values legend

- won_matches: Quantity of won matches against the same team.
  - Ex. England vs Germany are the teams, so last times England
defeated Germany.
- lost_matches: Quantity of lost matches against the same team.
  - Ex. England vs Germany are the teams, so last times England
was defeated by Germany.
- tie_matches: Quantity of tie matches against the same team.
  - Ex. England vs Germany are the teams, so last times England
and Germany had a tie.
- total_matches: Total quantity of matches against the same team.
  - Ex. England vs Germany are the teams, so quantity of matches counting won, lost, and tie matches.
- value_main_team: Price of the whole team.
  - Ex. England vs Germany are the teams, so the whole price of the England team.
- value_other_team: Price of the other team.
  - Ex. England vs Germany are the teams, so the whole price of the Germany team.
- yellow: Quantity of yellow cards, **POSSITIVE FOR THE MAIN TEAM NEGATIVE FOR THE OTHER TEAM**.
  - Ex. England vs Germany are the teams, so if England has a yellow card is equal 1, but if Germany has a yellow card is equal -1.
- red: Quantity of red cards, **POSSITIVE FOR THE MAIN TEAM NEGATIVE FOR THE OTHER TEAM**.
  - Ex. England vs Germany are the teams, so if England has a red card is equal 1, but if Germany has a red card is equal -1.
- percentage_team: Percentage of the main team is gonna play.
  - Ex. England vs Germany, so the percentage of the main team of England is gonna play if the whole main team is gonna play 1 if there are some main team players missing minus than 1.
- percentage_other_team: Percentage of the main team is gonna play.
  - Ex. England vs Germany, so the percentage of the main team of Germany is gonna play if the whole main team is gonna play 1 if there are some main team players missing minus than 1.
- iterations: Quantity of iterations for the simulation.

In [43]:
#@title Values
won_matches = 4 #@param {type:"integer"}
lost_matches = 1 #@param {type:"number"}
tie_matches = 7 #@param {type:"number"}
total_matches = 12 #@param {type:"number"}
value_main_team = 62230000 #@param {type:"number"}
value_other_team = 104100000 #@param {type:"number"}
yellow = 0 #@param {type:"integer"}
red = 0 #@param {type:"integer"}
percentage_team = 1 #@param {type:"number"}
percentage_other_team = 1 #@param {type:"number"}
iterations = 100000 #@param {type:"integer"}




In [44]:
#@title Betx
import random

class betx:

  def __init__(self, won_matches, lost_matches, tie_matches, total_matches, value_main_team, value_other_team, yellow, red, percentage_team, percentage_other_team, iterations):

    self.won_matches = won_matches
    self.lost_matches = lost_matches
    self.tie_matches = tie_matches
    self.total_matches = total_matches
    self.value_main_team = value_main_team
    self.value_other_team = value_other_team
    self.yellow = yellow
    self.red = red
    self.percentage_team = percentage_team
    self.percentage_other_team = percentage_other_team
    self.iterations = iterations

  def get_percentages(self):

    self.won_matches = self.won_matches/self.total_matches
    self.lost_matches = self.lost_matches/self.total_matches
    self.tie_matches = self.tie_matches/self.total_matches
	
    return self.won_matches, self.lost_matches, self.tie_matches

  def difference(self):

    p_diff = (self.value_main_team - self.value_other_team) / self.value_main_team

    return p_diff

  def bernoulli_dist(self):

    p1, p2, p3 = self.get_percentages()

    p1 = p1 * (1-p1)
    p2 = p2 * (1-p2)
    p3 = p3 * (1-p3)

    return p1, p2, p3

  def weights(self, w_diff= 0.5, w_diff_reward=0.02, w_diff_reward_tie=0.02, w_yellow_reward=0.02, w_red_reward=0.03, w_percentage_team_reward=0.01):

      p1, p2, p3 = self.bernoulli_dist()

      if self.difference() > w_diff:

        p1 += w_diff_reward

      elif self.difference() < -abs(w_diff):

        p2 += w_diff_reward

      else:

        p3 += w_diff_reward_tie

      if self.yellow < 0:

        p1 += abs(self.yellow) * w_yellow_reward

      elif self.yellow > 0:

        p2 += self.yellow * w_yellow_reward

      if self.red < 0:

        p1 += abs(self.red) * w_red_reward

      elif self.red > 0:

        p2 += self.red * w_red_reward

      if self.percentage_team < 1:

        p1 -= (1 - self.percentage_team) * w_percentage_team_reward

      elif self.percentage_other_team < 1:

        p2 -= (1 - self.percentage_other_team) * w_percentage_team_reward

      return p1, p2, p3

  def sample(self):

      weights = self.weights()
      items = ["win", "loose", "tie"]

      s = random.choices(items, weights, k=self.iterations)

      return s

  def simulation(self):

      win = 0
      loose = 0
      tie = 0

      for s in self.sample():

        if s == "win":

          win += 1

        elif s == "loose":

          loose += 1

        else:

          tie += 1

      win = win / len(self.sample())
      loose = loose / len(self.sample())
      tie = tie / len(self.sample())

      return win, loose, tie

  def abstract(self):

    p1, p2, p3 = self.bernoulli_dist()
    win, loose, tie = self.simulation()

    print("wins:", self.won_matches)
    print("lost:", self.lost_matches)
    print("tie:", self.tie_matches)
    print("Percentual difference:", self.difference())
    print('Bernoulli - Winning:', p1)
    print('Bernoulli - Looses:', p2)
    print('Bernoulli - Tie:', p3)
    print("Winning rate:", win)
    print("Loosing rate:", loose)
    print("Tie rate:", tie)
    

# Weight variables legend
- w_diff, w_diff_reward, w_diff_reward_tie:
  - w_diff is the margin of percentual difference to assign a reward, the default value is 0.5.
  - w_diff_reward is the reward for one or the other team due to the poderntual difference, the default value is 0.02.
  - w_diff_reward_tie is the reward in case there is not an acceptable percentual difference, the default value is 0.02.
```
if self.difference() > w_diff:

        p1 += w_diff_reward

      elif self.difference() < -abs(w_diff):

        p2 += w_diff_reward

      else:

        p3 += w_diff_reward_tie
```

- w_yellow_reward
  - This is the punishment in case a team has yellow cards, it is gonna assign a reward to the other team, the default value is 0.02.
```
if self.yellow < 0:

        p1 += abs(self.yellow) * w_yellow_reward

      elif self.yellow > 0:

        p2 += self.yellow * w_yellow_reward
```

- w_red_reward
  - This is the punishment in case a team has red cards, it is gonna assign a reward to the other team, the default value is 0.03.
```
if self.red < 0:

        p1 += abs(self.red) * w_red_reward

      elif self.red > 0:

        p2 += self.red * w_red_reward
```

- w_percentage_team_reward
  - This is a punishment in case there are persons of the main team missing, the default value is 0.01.
```
if self.percentage_team < 1:

        p1 -= (1 - self.percentage_team) * w_percentage_team_reward

      elif self.percentage_other_team < 1:

        p2 -= (1 - self.percentage_other_team) * w_percentage_team_reward

      return p1, p2, p3
```






In [45]:
#@title Twik weights variables
enable_cell = False #@param {type:"boolean"}
w_diff = 0 #@param {type:"number"}
w_diff_reward = 0 #@param {type:"number"}
w_diff_reward_tie = 0 #@param {type:"number"}
w_yellow_reward = 0 #@param {type:"number"}
w_red_reward = 0 #@param {type:"number"}
w_percentage_team_reward = 0 #@param {type:"number"}


In [46]:
#@title Twik weights

sim = betx(won_matches, lost_matches, tie_matches, total_matches, value_main_team, value_other_team, yellow, red, percentage_team, percentage_other_team, iterations)

if enable_cell == True:

  sim.weights(w_diff, w_diff_reward, w_diff_reward_tie, w_yellow_reward, w_red_reward, w_percentage_team_reward)

In [47]:
#@title Result abstract
sim.abstract()

wins: 1.6075102880658434e-05
lost: 4.0187757201646085e-06
tie: 2.813143004115226e-05
Percentual difference: -0.6728266109593444
Bernoulli - Winning: 0.22222222222222224
Bernoulli - Looses: 0.07638888888888888
Bernoulli - Tie: 0.24305555555555555
Winning rate: 0.2719
Loosing rate: 0.26638
Tie rate: 0.46172
